In [1]:
import sys
from os import path
import platform
import os
import pickle
os.environ['ARCHDEFS'] = '/home/wangpenghao/cutest/archdefs/'
os.environ['SIFDECODE'] = '/home/wangpenghao/cutest/sifdecode/'
os.environ['MASTSIF'] = '/home/wangpenghao/cutest/mastsif/'
os.environ['CUTEST'] = '/home/wangpenghao/cutest/cutest/'
os.environ['MYARCH'] = 'pc64.lnx.gfo'
import scipy
from scipy.optimize import rosen, rosen_der, rosen_hess, NonlinearConstraint
from scipy.optimize import minimize
from scipy.sparse import coo_array
import numpy as np
import pycutest
import pandas as pd
import time
import tqdm
from pdfo import pdfo

In [2]:
def load_data_from_pickle(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [3]:
# Input path
problem_path = "/home/wangpenghao/SOLcode/cutest/unconstrained_problems_Small.csv"
pdfo_direct_path = '/home/wangpenghao/SOLcode/cutest/pdfo_direct_small_res.pkl'
pdfo_key_path = '/home/wangpenghao/SOLcode/cutest/pdfo_small_results.pkl'
cg_direct_path = '/home/wangpenghao/SOLcode/cutest/cg_direct_small_res.pkl'
cg_key_path = '/home/wangpenghao/SOLcode/cutest/cg_small_results.pkl'
bfgs_direct_path = '/home/wangpenghao/SOLcode/cutest/bfgs_direct_small_res.pkl'
bfgs_key_path = '/home/wangpenghao/SOLcode/cutest/bfgs_small_results.pkl'
# Output path
nfev_success_path = '/home/wangpenghao/SOLcode/cutest/SMALL_results_nfev_success.xlsx'

In [4]:
problem = pd.read_csv(problem_path)
pdfo_direct_res = pd.DataFrame(load_data_from_pickle(pdfo_direct_path))
pdfo_key_res = pd.DataFrame(load_data_from_pickle(pdfo_key_path))
cg_direct_res = pd.DataFrame(load_data_from_pickle(cg_direct_path))
cg_key_res = pd.DataFrame(load_data_from_pickle(cg_key_path))
bfgs_direct_res = pd.DataFrame(load_data_from_pickle(bfgs_direct_path))
bfgs_key_res = pd.DataFrame(load_data_from_pickle(bfgs_key_path))
print(f"length of pdfo_direct_res: {len(pdfo_direct_res)}")
print(f"length of pdfo_res: {len(pdfo_key_res)}")
print(f"length of cg_direct_res: {len(cg_direct_res)}")
print(f"length of cg_res: {len(cg_key_res)}")
print(f"length of bfgs_direct_res: {len(bfgs_direct_res)}")
print(f"length of bfgs_res: {len(bfgs_key_res)}")

length of pdfo_direct_res: 172
length of pdfo_res: 172
length of cg_direct_res: 172
length of cg_res: 172
length of bfgs_direct_res: 172
length of bfgs_res: 172


In [5]:
print(pdfo_direct_res.columns)
print(cg_direct_res.columns)
print(bfgs_direct_res.columns)
print(pdfo_key_res.columns)
print(cg_key_res.columns)
print(bfgs_key_res.columns)

Index(['message', 'success', 'status', 'fun', 'x', 'nfev', 'fun_history',
       'method', 'warnings'],
      dtype='object')
Index(['fun', 'jac', 'nfev', 'njev', 'status', 'success', 'message', 'x',
       'nit', 'allvecs'],
      dtype='object')
Index(['fun', 'jac', 'hess_inv', 'nfev', 'njev', 'status', 'success',
       'message', 'x', 'nit', 'allvecs'],
      dtype='object')
Index(['Problem', 'Objective', 'Success', 'solution', 'Message', 'status',
       'nfev', 'fhist'],
      dtype='object')
Index(['Problem', 'Objective', 'Success', 'solution', 'Message', 'status',
       'nfev', 'niter'],
      dtype='object')
Index(['Problem', 'Objective', 'Success', 'solution', 'Message', 'status',
       'nfev', 'niter'],
      dtype='object')


## Collect Key Results

In [6]:
pdfo_key_res_suffix = pdfo_key_res.add_suffix('_pdfo')
cg_key_res_suffix = cg_key_res.add_suffix('_cg')
bfgs_key_res_suffix = bfgs_key_res.add_suffix('_bfgs')
# 合并DataFrame，现在不需要suffixes参数，因为列名已经是唯一的
df_merged = pd.merge(pdfo_key_res_suffix, cg_key_res_suffix, left_on='Problem_pdfo', right_on='Problem_cg', how='outer')
df_merged = pd.merge(df_merged, bfgs_key_res_suffix, left_on='Problem_pdfo', right_on='Problem_bfgs', how='outer')
# 创建 df_final 并确保它是一个独立副本
df_merged = pd.merge(df_merged, problem, left_on='Problem_pdfo', right_on='data', how='left')
#print(df_merged.head(1))
df_final = df_merged[['Problem_pdfo', 'n', 'Objective_pdfo', 'Objective_cg', 'Objective_bfgs', 'nfev_pdfo', 'nfev_cg', 'nfev_bfgs', 'Success_pdfo', 'Success_cg', 'Success_bfgs','niter_cg','niter_bfgs']].copy()
df_final.rename(columns={'Problem_pdfo': 'Problem', 'n': 'Dimension'}, inplace=True)
print(df_final.head(1))
def highlight_min(s):
    is_min = s == s.min()
    return ['background-color: red' if v else '' for v in is_min]

# 应用样式
df_styled = df_final.style.apply(highlight_min, axis=1, subset=['Objective_pdfo', 'Objective_cg', 'Objective_bfgs'])
df_styled = df_styled.format({
    'Objective_pdfo': '{:.5e}',
    'Objective_cg': '{:.5e}',
    'Objective_bfgs': '{:.5e}'
})
with pd.ExcelWriter(nfev_success_path, engine='openpyxl') as writer:
    df_styled.to_excel(writer, index=False)

  Problem  Dimension  Objective_pdfo  Objective_cg  Objective_bfgs  nfev_pdfo  \
0   AKIVA          2        6.166042      6.166042        6.166042         74   

   nfev_cg  nfev_bfgs  Success_pdfo  Success_cg  Success_bfgs  niter_cg  \
0       92         45          True        True          True         8   

   niter_bfgs  
0          11  
